In [1]:
import pandas as pd
import numpy as np
import itertools

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [21]:
!ls

README.md                 constituents_csv.csv      stock-fraud-predict.ipynb
all_stocks_5yr.csv        scac.csv


In [108]:
df = pd.read_csv('scac.csv')

In [109]:
df.head()

,filing_name,filing_date,district_court,exchange,ticker
0,"ILG, Inc.",07/06/2018,D. Delaware,NASDAQ,ILG
1,KLX Inc.,07/06/2018,D. Delaware,NASDAQ,KLXI
2,Triangle Capital Corporation,07/06/2018,E.D. North Carolina,New York SE,TCAP
3,Vectren Corporation,07/05/2018,S.D. Indiana,New York SE,VVC
4,DCT Industrial Trust Inc.,07/02/2018,D. Colorado,New York SE,DCT


In [110]:
df1 = pd.read_csv('all_stocks_5yr.csv')

In [111]:
df1.head()

,date,open,high,low,close,volume,Name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL


In [112]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 619040 entries, 0 to 619039
Data columns (total 7 columns):
date      619040 non-null object
open      619029 non-null float64
high      619032 non-null float64
low       619032 non-null float64
close     619040 non-null float64
volume    619040 non-null int64
Name      619040 non-null object
dtypes: float64(4), int64(1), object(2)
memory usage: 33.1+ MB


In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4996 entries, 0 to 4995
Data columns (total 5 columns):
filing_name       4996 non-null object
filing_date       4996 non-null object
district_court    4996 non-null object
exchange          4996 non-null object
ticker            4996 non-null object
dtypes: object(5)
memory usage: 195.2+ KB


In [114]:
df1.rename(columns={'Name':'ticker'}, inplace=True)

In [115]:
df1.head()

,date,open,high,low,close,volume,ticker
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL


In [116]:
df['sued'] = 1

In [117]:
df.head()

,filing_name,filing_date,district_court,exchange,ticker,sued
0,"ILG, Inc.",07/06/2018,D. Delaware,NASDAQ,ILG,1
1,KLX Inc.,07/06/2018,D. Delaware,NASDAQ,KLXI,1
2,Triangle Capital Corporation,07/06/2018,E.D. North Carolina,New York SE,TCAP,1
3,Vectren Corporation,07/05/2018,S.D. Indiana,New York SE,VVC,1
4,DCT Industrial Trust Inc.,07/02/2018,D. Colorado,New York SE,DCT,1


In [118]:
#df_merge = pd.merge(df, df1, on='ticker')
df_merged = df1.merge(df,how='left', left_on='ticker', right_on='ticker')

In [119]:
df.head()

,filing_name,filing_date,district_court,exchange,ticker,sued
0,"ILG, Inc.",07/06/2018,D. Delaware,NASDAQ,ILG,1
1,KLX Inc.,07/06/2018,D. Delaware,NASDAQ,KLXI,1
2,Triangle Capital Corporation,07/06/2018,E.D. North Carolina,New York SE,TCAP,1
3,Vectren Corporation,07/05/2018,S.D. Indiana,New York SE,VVC,1
4,DCT Industrial Trust Inc.,07/02/2018,D. Colorado,New York SE,DCT,1


In [120]:
df1.head()

,date,open,high,low,close,volume,ticker
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL


In [121]:
df_merged.head()

,date,open,high,low,close,volume,ticker,filing_name,filing_date,district_court,exchange,sued
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL,NaN,NaN,NaN,NaN,NaN
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL,NaN,NaN,NaN,NaN,NaN
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL,NaN,NaN,NaN,NaN,NaN
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL,NaN,NaN,NaN,NaN,NaN
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL,NaN,NaN,NaN,NaN,NaN


In [122]:
df_merged.sued.unique()

array([nan,  1.])

In [123]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 837587 entries, 0 to 837586
Data columns (total 12 columns):
date              837587 non-null object
open              837574 non-null float64
high              837577 non-null float64
low               837577 non-null float64
close             837587 non-null float64
volume            837587 non-null int64
ticker            837587 non-null object
filing_name       519028 non-null object
filing_date       519028 non-null object
district_court    519028 non-null object
exchange          519028 non-null object
sued              519028 non-null float64
dtypes: float64(5), int64(1), object(6)
memory usage: 83.1+ MB


In [52]:
def notSued(x):
    if type(x) is float64:
        return x
    elif x:
        return 0
    else:
        return

for i in df_merged['sued']
    if isNull(i) = True:
        df_merged['sued']=0
        
i for null(i) in df_merged['sued']

In [124]:
df_merged['sued'] = np.where(df_merged['sued']==1, 1, 0)

In [125]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 837587 entries, 0 to 837586
Data columns (total 12 columns):
date              837587 non-null object
open              837574 non-null float64
high              837577 non-null float64
low               837577 non-null float64
close             837587 non-null float64
volume            837587 non-null int64
ticker            837587 non-null object
filing_name       519028 non-null object
filing_date       519028 non-null object
district_court    519028 non-null object
exchange          519028 non-null object
sued              837587 non-null int64
dtypes: float64(4), int64(2), object(6)
memory usage: 83.1+ MB


In [99]:
df_merged.drop(columns='sued')

,date,open,high,low,close,volume,ticker,filing_name,filing_date,district_court,exchange,sued1
0,2013-02-08,15.07,15.1200,14.6300,14.75,8407500,AAL,NaN,NaN,NaN,NaN,0
1,2013-02-11,14.89,15.0100,14.2600,14.46,8882000,AAL,NaN,NaN,NaN,NaN,0
2,2013-02-12,14.45,14.5100,14.1000,14.27,8126000,AAL,NaN,NaN,NaN,NaN,0
3,2013-02-13,14.30,14.9400,14.2500,14.66,10259500,AAL,NaN,NaN,NaN,NaN,0
4,2013-02-14,14.94,14.9600,13.1600,13.99,31879900,AAL,NaN,NaN,NaN,NaN,0
5,2013-02-15,13.93,14.6100,13.9300,14.50,15628000,AAL,NaN,NaN,NaN,NaN,0
6,2013-02-19,14.33,14.5600,14.0800,14.26,11354400,AAL,NaN,NaN,NaN,NaN,0
7,2013-02-20,14.17,14.2600,13.1500,13.33,14725200,AAL,NaN,NaN,NaN,NaN,0
8,2013-02-21,13.62,13.9500,12.9000,13.37,11922100,AAL,NaN,NaN,NaN,NaN,0
9,2013-02-22,13.57,13.6000,13.2100,13.57,6071400,AAL,NaN,NaN,NaN,NaN,0


In [100]:
df_merged.rename(columns={'sued1': 'sued'}, inplace=True)

In [101]:
df_merged.exchange.value_counts()

New York SE         378430
NASDAQ              135562
American SE           2518
Privately Traded      1259
ETF                   1259
Name: exchange, dtype: int64

In [102]:
df_merged.filing_name.value_counts()

Juniper Networks, Inc.                                                                                               6295
Chesapeake Energy Corporation                                                                                        5036
General Electric Company                                                                                             5036
Medtronic, Inc.                                                                                                      5036
Advanced Micro Devices, Inc.                                                                                         3777
Merck & Co., Inc.                                                                                                    3777
Aetna Inc.                                                                                                           3777
American Express Company                                                                                             3777
Johnson & Johnson       

In [127]:
from datetime import datetime

In [128]:
df_merged.filing_date = pd.to_datetime(df_merged['filing_date'])

In [130]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 837587 entries, 0 to 837586
Data columns (total 12 columns):
date              837587 non-null object
open              837574 non-null float64
high              837577 non-null float64
low               837577 non-null float64
close             837587 non-null float64
volume            837587 non-null int64
ticker            837587 non-null object
filing_name       519028 non-null object
filing_date       519028 non-null datetime64[ns]
district_court    519028 non-null object
exchange          519028 non-null object
sued              837587 non-null int64
dtypes: datetime64[ns](1), float64(4), int64(2), object(5)
memory usage: 83.1+ MB


In [131]:
df_sectors = pd.read_csv('constituents_csv.csv')

In [132]:
df_sectors.head()

,Symbol,Name,Sector
0,MMM,3M Company,Industrials
1,AOS,A.O. Smith Corp,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie Inc.,Health Care
4,ACN,Accenture plc,Information Technology


In [133]:
df_sectors.rename(columns={'Symbol':'ticker'}, inplace=True)

In [134]:
df_sectors.head()

,ticker,Name,Sector
0,MMM,3M Company,Industrials
1,AOS,A.O. Smith Corp,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie Inc.,Health Care
4,ACN,Accenture plc,Information Technology


In [135]:
df_merged2 = df_merged.merge(df_sectors,how='left', left_on='ticker', right_on='ticker')
#df_merge2 = pd.merge(df_merge, df_sectors, on='ticker')

In [137]:
df_merged2.head()

,date,open,high,low,close,volume,ticker,filing_name,filing_date,district_court,exchange,sued,Name,Sector
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL,NaN,NaT,NaN,NaN,0,American Airlines Group,Industrials
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL,NaN,NaT,NaN,NaN,0,American Airlines Group,Industrials
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL,NaN,NaT,NaN,NaN,0,American Airlines Group,Industrials
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL,NaN,NaT,NaN,NaN,0,American Airlines Group,Industrials
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL,NaN,NaT,NaN,NaN,0,American Airlines Group,Industrials


In [138]:
df_merged2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 837587 entries, 0 to 837586
Data columns (total 14 columns):
date              837587 non-null object
open              837574 non-null float64
high              837577 non-null float64
low               837577 non-null float64
close             837587 non-null float64
volume            837587 non-null int64
ticker            837587 non-null object
filing_name       519028 non-null object
filing_date       519028 non-null datetime64[ns]
district_court    519028 non-null object
exchange          519028 non-null object
sued              837587 non-null int64
Name              822479 non-null object
Sector            822479 non-null object
dtypes: datetime64[ns](1), float64(4), int64(2), object(7)
memory usage: 95.9+ MB


In [140]:
df_merged2.Sector.value_counts()

Financials                    137479
Health Care                   130185
Information Technology        122571
Consumer Discretionary        121198
Industrials                   101845
Consumer Staples               47152
Energy                         41699
Utilities                      41537
Real Estate                    41531
Materials                      30987
Telecommunication Services      6295
Name: Sector, dtype: int64

In [141]:
df_sector = pd.get_dummies(df_merged2['Sector'])

In [142]:
df_final = pd.concat([df_merged2, df_sector], axis=1)

In [143]:
df_final.columns

Index(['date', 'open', 'high', 'low', 'close', 'volume', 'ticker',
       'filing_name', 'filing_date', 'district_court', 'exchange', 'sued',
       'Name', 'Sector', 'Consumer Discretionary', 'Consumer Staples',
       'Energy', 'Financials', 'Health Care', 'Industrials',
       'Information Technology', 'Materials', 'Real Estate',
       'Telecommunication Services', 'Utilities'],
      dtype='object')

In [145]:
df_final.head()

,date,open,high,low,close,volume,ticker,filing_name,filing_date,district_court,...,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Telecommunication Services,Utilities
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL,NaN,NaT,NaN,...,0,0,0,0,1,0,0,0,0,0
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL,NaN,NaT,NaN,...,0,0,0,0,1,0,0,0,0,0
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL,NaN,NaT,NaN,...,0,0,0,0,1,0,0,0,0,0
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL,NaN,NaT,NaN,...,0,0,0,0,1,0,0,0,0,0
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL,NaN,NaT,NaN,...,0,0,0,0,1,0,0,0,0,0


In [146]:
df_final.district_court.value_counts()

S.D. New York          117356
N.D. California         63481
D. New Jersey           27397
C.D. California         25761
N.D. Illinois           22662
D. Massachusetts        22662
D. Minnesota            18885
D. Connecticut          16367
S.D. Ohio               12590
E.D. Missouri           11331
E.D. Michigan           11331
D. Colorado             10369
N.D. Georgia            10072
D. Delaware              9773
E.D. Pennsylvania        8813
S.D. California          8813
M.D. Tennessee           7851
W.D. Washington          7554
S.D. Florida             7554
N.D. Texas               7464
N.D. Ohio                6295
E.D. New York            6295
W.D. Oklahoma            5036
D. Nevada                4338
E.D. Virginia            4305
S.D. Indiana             3777
D. Rhode Island          3777
E.D. Tennessee           3777
S.D. Texas               3777
W.D. Kentucky            3777
W.D. Missouri            3777
D. Oregon                2518
N.D. Alabama             2518
M.D. Flori

In [147]:
df_final.head()

,date,open,high,low,close,volume,ticker,filing_name,filing_date,district_court,...,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Telecommunication Services,Utilities
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL,NaN,NaT,NaN,...,0,0,0,0,1,0,0,0,0,0
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL,NaN,NaT,NaN,...,0,0,0,0,1,0,0,0,0,0
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL,NaN,NaT,NaN,...,0,0,0,0,1,0,0,0,0,0
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL,NaN,NaT,NaN,...,0,0,0,0,1,0,0,0,0,0
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL,NaN,NaT,NaN,...,0,0,0,0,1,0,0,0,0,0


In [149]:
df_final.describe()

,open,high,low,close,volume,sued,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Telecommunication Services,Utilities
count,837574.000000,837577.000000,837577.000000,837587.000000,8.375870e+05,837587.000000,837587.000000,837587.000000,837587.000000,837587.000000,837587.000000,837587.000000,837587.000000,837587.000000,837587.000000,837587.000000,837587.000000
mean,83.500098,84.267744,82.720669,83.519700,6.110539e+06,0.619671,0.144699,0.056295,0.049785,0.164137,0.155429,0.121593,0.146338,0.036996,0.049584,0.007516,0.049591
std,106.820307,107.744581,105.852739,106.831904,1.303700e+07,0.485468,0.351797,0.230491,0.217500,0.370400,0.362313,0.326816,0.353445,0.188751,0.217084,0.086366,0.217099
min,1.620000,1.690000,1.500000,1.590000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,37.710000,38.070000,37.340000,37.705000,1.197900e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,60.460000,61.030000,59.900000,60.490000,2.456047e+06,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,93.290000,94.120000,92.490000,93.330000,5.500547e+06,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2044.000000,2067.990000,2035.110000,2049.000000,6.182376e+08,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
